In [1]:
import numpy as np
import time
import datetime as dt
import yfinance as yf
from ib_insync import *
import pandas as pd
import sys
import sched
import datetime
import pandas_ta as ta
import warnings

##### Warnings for invalid values/dividing by zero
warnings.filterwarnings("ignore")

def exit():
    print('{} Now the system will exit '.format(datetime.datetime.now()))
    sys.exit()

class CaptureReturn:
    def __init__(self, func, initial_state):
        self.func = func
        self.state = initial_state

    def call_func(self):
        self.output = self.func(self.state)
        return self.output

def get_stock_info(stock): 
    attempts = 3 
    for attempt in range(1, attempts+1): 
        
        data = yf.download(tickers=stock, period = '2d', interval='2m').round(decimals=3)
        if data.empty:
            print(f'Attempt {attempt} Failed, Dataset Empty.')
            time.sleep(3)
            continue
        else:
            print(f'Downloaded in Attempt {attempt}')
            break      
    return data

def place_order(direction, stock, amount): 
    order = MarketOrder(direction, amount)
    contract = Stock(stock, 'SMART', 'USD')
    trade = ib.placeOrder(contract, order)         
    return trade

def order_status(trade): ################
    if trade.orderStatus.status == 'Filled': ################
        fill = trade.fills[-1] ################
        price = fill.execution.avgPrice ################
        print(f'{fill.time} - {fill.execution.avgPrice}')
        return price ################

def trade_algo(data, buy_signals, sell_signals, buy_price, trigger, stock, amount): ################

    def ema(dataset, price, interval):
        ema_column = ta.ema(dataset[price], length=interval)
        data_ema = dataset.assign(ema=ema_column)
        return data_ema
    
    def atr(dataset, high, low, close, interval):
        average_tr = ta.atr(dataset[high], dataset[low], dataset[close])
        data_atr = dataset.assign(atr = average_tr)
        return data_atr
    
    df_ema_12 = ema(data, 'Close', 12).round(decimals=3)
    df_ema_50 = ema(data, 'Close', 50).round(decimals=3)
    df_atr = atr(data, 'High', 'Low', 'Close', 14).round(decimals=3)
    
    high = data["High"]
    open = data["Open"]
    close = data["Close"]
    low = data["Low"]
    atr = df_atr["atr"]
    ema_12 = df_ema_12["ema"]
    ema_50 = df_ema_50["ema"]
    
    body_candle_1 = close[-3] - open[-3]
    body_candle = close[-2] - open[-2]

    high_stick_hammer = abs(high[-2] - close[-2])
    low_stick_hammer = abs(open[-2] - low[-2])

    high_stick_int = abs(high[-2] - open[-2])
    low_stick_int = abs(close[-2] - low[-2])
                
    ##### Hammer Candlestick patterns    
        
    if low_stick_hammer/abs(body_candle) > 3 and body_candle > 0 \
    and high_stick_hammer/low_stick_hammer < 0.25 and ema_12[-2] > ema_50[-2] and trigger != 1:

        trade = place_order('BUY', stock, amount)
        buy_price = close[-2]
        print("bought at ", close[-2], "at ", data.index[-2], 'time now: ',
                  datetime.datetime.now().strftime("%H:%M:%S"), 'with hammer pattern')
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "low stick/body: ", 
              low_stick_hammer/abs(body_candle), "high stick/low stick", high_stick_hammer/low_stick_hammer)
        time.sleep(3) 
        buy_signals.append(close[-2])
        trigger = 1
    
    ##### Int Hammer Candlestick patterns    

    elif low_stick_int/abs(body_candle) > 3 and body_candle < 0 \
    and high_stick_int/low_stick_int < 0.25 and ema_12[-2] > ema_50[-2] and trigger != 1:

        trade = place_order('BUY', stock, amount)
        buy_price = close[-2]
        print("bought at ", close[-2], "at ", data.index[-2], 'time now: ',
                  datetime.datetime.now().strftime("%H:%M:%S"), 'with int hammer pattern')
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "low stick/body: ", 
              low_stick_int/abs(body_candle), "high stick/low stick", high_stick_int/low_stick_int)

        time.sleep(3) 
        buy_signals.append(close[-2])
        trigger = 1
    
    ##### Bullish/Bearing Engulfing patterns
       
    elif open[-2] <= close[-3] and close[-2] >= open[-3] \
        and abs(body_candle)/abs(body_candle_1) > 4 and body_candle_1 < 0 and body_candle > 0 \
        and close[-2]/open[-2] > 1.03 and trigger != 1:
            
            print(close[price]/open[price])
            print("engulfing pattern")
            buy_signals.append(close[price])
            sell_signals.append(float('nan'))
            profit.append(close[price])
            buy_price = close[price]
            trigger = 1
        
    ########## ATR sell off
    
    elif close[-2] > buy_price + 5*atr[-2] and trigger != -1 \
    or close[-2] < buy_price - 5*atr[-2] and trigger != -1:
        
        if trigger == 0:
            print("closing price: ", close[-2], "closing time: ", data.index[-2], "time now ", 
                  datetime.datetime.now().strftime("%H:%M:%S"))
            print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "time now: ",
                  datetime.datetime.now().strftime("%H:%M:%S"))
        
        else:
            trade = place_order('SELL', stock, amount)
            print("sold at: ", close[-2], "closing time: ", data.index[-2], "time now ", 
                  datetime.datetime.now().strftime("%H:%M:%S"))
            
            time.sleep(3)
            sell_signals.append(close[-2])
            trigger = -1
            
    else:
        print("closing price: ", close[-2], "closing time: ", data.index[-2], "time now ", 
              datetime.datetime.now().strftime("%H:%M:%S"))
        print("open: ", open[-2], "high: ", high[-2], "low: ", low[-2], "time now: ",
                  datetime.datetime.now().strftime("%H:%M:%S"))
        
    return buy_signals, sell_signals, buy_price, trigger

def save_csv_file(buy, sell, stock): ################
    ################
    dataset = pd.DataFrame({'BUY': buy, 'SELL': sell,}, columns=[stock,'BUY','SELL'])
    ################
    myfile = dataset.to_csv('C:/Users/shong/Desktop/trade_logs.csv', mode='a', index=False)
    return myfile

def duration(start_hour,start_min,end_hour,end_min):
    time_1 = datetime.timedelta(hours= start_hour, minutes=start_min, seconds=0)
    time_2 = datetime.timedelta(hours= end_hour, minutes=end_min, seconds=0)
    
    time_interval = (time_2 - time_1).total_seconds() + \
                        datetime.timedelta(days=1).total_seconds()
    time_interval = (time_interval/60)/2
    return time_interval

print("functions initialised")

functions initialised


In [2]:
def main():

    #stock = sys.argv[1] 
    #trade_amount = sys.argv[2]
    
    print("establishing connection...")
    util.startLoop()
    global ib
    ib = IB()
    ib.connect(host='127.0.0.1', port=7497, clientId=3)
    print("connection established")   
    
    stock = 'BTU'
    print("Stocks:", stock)
         
    buy_signals = []
    sell_signals = []
    buy_price = 0
    amount = 50
    minute = 42
    trigger = 0
    
    iterations = int(duration(22, minute, 4, 0)) 
    
    scheduler = sched.scheduler(time.time, time.sleep)

    capture_return = CaptureReturn(get_stock_info, stock)
    
    time_now = time.time()

    while True:
        for count in range(iterations-1):
            scheduler.enterabs(time_now + 120*(count+1), 1, capture_return.call_func)
            data = capture_return.call_func()
            
            if data.empty:
                print("Dataset not found at ", datetime.datetime.now().strftime("%H:%M:%S"))
                continue
            
            buy_signals, sell_signals, buy_price, trigger = trade_algo(data, buy_signals, 
                                                            sell_signals, buy_price, trigger, stock, amount)
            scheduler.run()

        if trigger == 1:
            time.sleep(80) ##################
            
            place_order('SELL', stock, amount)
            print("sold :) at market close ", data['Close'][-1])
            
            time.sleep(3) #############
            sell_signals.append(data['Close'][-1])
            time.sleep(3)
            file = save_csv_file(buy_signals, sell_signals, stock)
            time.sleep(3)
            ib.disconnect()
            quit(0)
            time.sleep(3)
            exit()
        else:
            file = save_csv_file(buy_signals, sell_signals, stock)
            time.sleep(3)
            ib.disconnect()
            quit(0)
            time.sleep(3)
            exit()

if __name__ == "__main__":
    main()

establishing connection...
connection established
Stocks: BTU
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.44 closing time:  2022-09-28 10:40:00-04:00 time now  22:42:31
open:  24.27 high:  24.48 low:  24.255 time now:  22:42:31
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.51 closing time:  2022-09-28 10:42:00-04:00 time now  22:44:30
open:  24.453 high:  24.56 low:  24.41 time now:  22:44:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.65 closing time:  2022-09-28 10:44:00-04:00 time now  22:46:30
open:  24.57 high:  24.65 low:  24.45 time now:  22:46:30
[*********************100%***********************] 

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.58 closing time:  2022-09-28 11:30:00-04:00 time now  23:32:30
open:  24.89 high:  24.93 low:  24.58 time now:  23:32:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.52 closing time:  2022-09-28 11:32:00-04:00 time now  23:34:30
open:  24.61 high:  24.633 low:  24.51 time now:  23:34:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.52 closing time:  2022-09-28 11:34:00-04:00 time now  23:36:30
open:  24.55 high:  24.57 low:  24.45 time now:  23:36:30
[********************

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.35 closing time:  2022-09-28 12:20:00-04:00 time now  00:22:30
open:  24.22 high:  24.35 low:  24.215 time now:  00:22:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.33 closing time:  2022-09-28 12:22:00-04:00 time now  00:24:30
open:  24.34 high:  24.34 low:  24.307 time now:  00:24:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.23 closing time:  2022-09-28 12:24:00-04:00 time now  00:26:30
open:  24.312 high:  24.312 low:  24.23 time now:  00:26:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*****************

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.58 closing time:  2022-09-28 14:00:00-04:00 time now  02:02:30
open:  24.565 high:  24.592 low:  24.54 time now:  02:02:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.57 closing time:  2022-09-28 14:02:00-04:00 time now  02:04:30
open:  24.56 high:  24.59 low:  24.56 time now:  02:04:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.585 closing time:  2022-09-28 14:04:00-04:00 time now  02:06:30
open:  24.6 high:  24.62 low:  24.57 time now:  02:06:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*******************

[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  25.005 closing time:  2022-09-28 15:40:00-04:00 time now  03:42:30
open:  24.935 high:  25.01 low:  24.924 time now:  03:42:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  25.05 closing time:  2022-09-28 15:42:00-04:00 time now  03:44:30
open:  25.01 high:  25.08 low:  25.0 time now:  03:44:30
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
[*********************100%***********************]  1 of 1 completed
Downloaded in Attempt 1
closing price:  24.995 closing time:  2022-09-28 15:44:00-04:00 time now  03:46:30
open:  25.05 high:  25.052 low:  24.96 time now:  03:46:30
[*****************

SystemExit: 